In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

import csv
import os

from evaluator import Evaluator

from config import OPUS_MT_MODELS, prefixes
from utils.data_utils import get_bbt_data
from utils.model_utils import load_baseline, load_model, translate_all

Loading config. model overview: {'da': 'Helsinki-NLP/opus-mt-en-gem', 'de': 'Helsinki-NLP/opus-mt-en-gem', 'no': 'Helsinki-NLP/opus-mt-en-gem', 'is': 'Helsinki-NLP/opus-mt-en-gem', 'lt': 'Helsinki-NLP/opus-mt-en-bat', 'pl': 'Helsinki-NLP/opus-mt-en-zlw', 'hu': 'Helsinki-NLP/opus-mt-en-urj', 'fr': 'Helsinki-NLP/opus-mt-en-roa', 'ms': 'Helsinki-NLP/opus-mt-en-map', 'sq': 'Helsinki-NLP/opus-mt-en-sq', 'eu': 'Helsinki-NLP/opus-mt-en-eu'}


In [3]:
lang = "no"
src_texts, tar_texts, prefixed = get_bbt_data(prefix=prefixes.get(lang, ""))
baseline_model, baseline_tokenizer = load_baseline(OPUS_MT_MODELS[lang])
baseline_translations = translate_all(prefixed, baseline_model, baseline_tokenizer, batch_size=16)
del baseline_model, baseline_tokenizer

100%|██████████| 6/6 [00:02<00:00,  2.54it/s]


In [4]:
baseline_translations[:5]

['Kan jeg mate jordnøtter til elefantene på bryllupet ditt?',
 'Selvfølgelig vil det være elefanter det er en stereotype som du mater dem jordnøtter',
 'Takk Penny og Leonard jeg var litt håpet jeg kunne sitte ved siden av Anu',
 'Ja det er bare vi ikke har gjort det ennå og jeg tror det er viktig å være sikker på at vi er kompatibel før vi gifter oss',
 'Jeg kan skaffe oss et rom på hotellet mitt.']

In [5]:
evaluator = Evaluator()
baseline_metrics = evaluator.get_results(
    predictions=baseline_translations,
    references=tar_texts,
    source=src_texts,
    lang=lang,
)

[nltk_data] Downloading package wordnet to /home/tollef/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/tollef/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/tollef/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
84it [00:14,  5.77it/s]


In [6]:
baseline_map = {}
for _source, _results in baseline_metrics.items():
    baseline_map[_source] = _results["prediction"]
baseline_map['can I feed peanuts to the elephants at your wedding?']


'Kan jeg mate jordnøtter til elefantene på bryllupet ditt?'

In [7]:
def print_results(metrics, n=5):
    c = 0
    for _source, _results in metrics.items():
        print(f"Source: {_source}")
        print(f"Target: {_results['target']}")
        print(f"Pred: {_results['prediction']}")
        print(f"Metrics: {_results['metrics']}")
        print()
        c += 1
        if c >= n:
            break
    
print_results(baseline_metrics)

Source: can I feed peanuts to the elephants at your wedding?
Target: Får jeg gi elefantene peanøtter?
Pred: Kan jeg mate jordnøtter til elefantene på bryllupet ditt?
Metrics: {'bleu': 5.52, 'r1': 0.35, 'r2': 0.0, 'rl': 0.24, 'chrF': 47.23, 'chrf++': 40.94, 'meteor': 0.23, 'bert_f1': 0.79, 'length': 57.0, 'len_ratio': 1.78}

Source: of course there'll be elephants it's a stereotype that you feed them peanuts
Target: Jo, men de får ingen peanøtter.
Pred: Selvfølgelig vil det være elefanter det er en stereotype som du mater dem jordnøtter
Metrics: {'bleu': 0.0, 'r1': 0.08, 'r2': 0.0, 'rl': 0.08, 'chrF': 20.61, 'chrf++': 15.46, 'meteor': 0.0, 'bert_f1': 0.68, 'length': 84.0, 'len_ratio': 2.71}

Source: thank you Penny and Leonard I was kind of hoping I could sit next to Anu
Target: Jeg vil sitte med Anu, Leonard.
Pred: Takk Penny og Leonard jeg var litt håpet jeg kunne sitte ved siden av Anu
Metrics: {'bleu': 3.46, 'r1': 0.36, 'r2': 0.0, 'rl': 0.27, 'chrF': 31.53, 'chrf++': 27.68, 'meteor'

# Model type 1

In [8]:
from config import COMPRESSION_RATIOS
evaluator = Evaluator()
MODEL_DIR = "OUTPUT"
lang = "no"

model1_results = {}

for comp in COMPRESSION_RATIOS:
    print(f"Compression ratio: {comp}")
    filename = f"BBT/comp_{comp}_results.csv"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    print(filename)
    with open(filename, "w", newline="") as f:
        model, tokenizer = load_model(
            path=f"../{MODEL_DIR}/en-{lang}/{comp}"
        )
        translated = translate_all(prefixed, model, tokenizer, batch_size=16)
        _results = evaluator.get_results(
            predictions=translated,
            references=tar_texts,
            source=src_texts,
            lang=lang,
        )

        model1_results[comp] = _results

[nltk_data] Downloading package wordnet to /home/tollef/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/tollef/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/tollef/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Compression ratio: 0.5
BBT/comp_0.5_results.csv


100%|██████████| 6/6 [00:01<00:00,  5.25it/s]
84it [00:14,  5.94it/s]


Compression ratio: 0.6
BBT/comp_0.6_results.csv


100%|██████████| 6/6 [00:01<00:00,  3.82it/s]
84it [00:12,  6.76it/s]


...

100%|██████████| 6/6 [00:01<00:00,  3.30it/s]
84it [00:11,  7.02it/s]


Compression ratio: 0.9
BBT/comp_0.9_results.csv


100%|██████████| 6/6 [00:01<00:00,  3.15it/s]
84it [00:11,  7.30it/s]


Compression ratio: 1.0
BBT/comp_1.0_results.csv


100%|██████████| 6/6 [00:01<00:00,  3.24it/s]
84it [00:11,  7.34it/s]


In [93]:
for comp, results in model1_results.items():
    print(f"Compression ratio: {comp}")
    print_results(results)
    print("_"*40)

Compression ratio: 0.5
Source: can I feed peanuts to the elephants at your wedding?
Target: Får jeg gi elefantene peanøtter?
Pred: Kan jeg gi elefanter jordnøtter?
Metrics: {'bleu': 17.97, 'r1': 0.46, 'r2': 0.18, 'rl': 0.46, 'chrF': 61.21, 'chrf++': 54.65, 'meteor': 0.43, 'bert_f1': 0.9, 'length': 32.0, 'len_ratio': 1.0}

Source: of course there'll be elephants it's a stereotype that you feed them peanuts
Target: Jo, men de får ingen peanøtter.
Pred: Det er en stereotype du mater peanuts.
Metrics: {'bleu': 5.52, 'r1': 0.0, 'r2': 0.0, 'rl': 0.0, 'chrF': 16.91, 'chrf++': 14.25, 'meteor': 0.06, 'bert_f1': 0.73, 'length': 38.0, 'len_ratio': 1.23}

Source: thank you Penny and Leonard I was kind of hoping I could sit next to Anu
Target: Jeg vil sitte med Anu, Leonard.
Pred: Takk, Penny og Leonard.
Metrics: {'bleu': 12.87, 'r1': 0.2, 'r2': 0.0, 'rl': 0.2, 'chrF': 26.91, 'chrf++': 27.01, 'meteor': 0.33, 'bert_f1': 0.73, 'length': 23.0, 'len_ratio': 0.74}

Source: yeah it's just we haven't done

In [84]:
model1_results.keys()

dict_keys([0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

In [94]:
import pandas as pd
from collections import defaultdict
from IPython.display import display

included_compressions = [0.5, 0.7, 1.0]
interesting_sentences = [
    "can I feed peanuts to the elephants at your wedding?",
    "you realise we still have a lot of work to do before we can publish",
    "It's kinda beautiful isn't it?"
]
interesting_sentences = [sentence.lower() for sentence in interesting_sentences]

gathered_data = defaultdict(list)
for comp, results in model1_results.items():
    if comp not in included_compressions:
        continue
    
    for _source, _results in results.items():
        if _source.lower() not in interesting_sentences:
            continue
        print(_source)
            
        pred = _results["prediction"]
        # gathered_data[sample_sizes][comp].append(pred)
        gathered_data[(_source, _results["target"])].append((comp, pred))

for (source, target), data in gathered_data.items():
    print(source)
    print(target)
    df = pd.DataFrame(data).T
    df.columns = included_compressions
    # ensure to display the full width of the text:
    # pd.set_option('display.max_colwidth', None)
    display(df)
    # print("_"*40)

can I feed peanuts to the elephants at your wedding?
you realise we still have a lot of work to do before we can publish
It's kinda beautiful isn't it?
can I feed peanuts to the elephants at your wedding?
you realise we still have a lot of work to do before we can publish
It's kinda beautiful isn't it?
can I feed peanuts to the elephants at your wedding?
you realise we still have a lot of work to do before we can publish
It's kinda beautiful isn't it?
can I feed peanuts to the elephants at your wedding?
Får jeg gi elefantene peanøtter?


,0.5,0.7,1.0
0,0.5,0.7,1.0
1,Kan jeg gi elefanter jordnøtter?,Kan jeg gi elefantene peanøtter?,Kan jeg gi jordnøtter til elefantene på bryllupet deres?


you realise we still have a lot of work to do before we can publish
Det er fremdeles mye som gjenstår.


,0.5,0.7,1.0
0,0.5,0.7,1.0
1,Vi har mye å gjøre før vi kan publisere.,Vi har mye å gjøre før vi kan publisere.,Vi har fortsatt mye å gjøre før vi kan publisere.


It's kinda beautiful isn't it?
Det er jo vakkert.


,0.5,0.7,1.0
0,0.5,0.7,1.0
1,Er det ikke vakkert?,Er det ikke vakkert?,"Det er vakkert, ikke sant?"
